In [1]:
# Load env variables and create client
import base64
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()

client = Anthropic()
model = "claude-sonnet-4-5"

In [2]:
# Helper functions
from anthropic.types import Message


def add_user_message(messages, message):
    user_message = {
        "role": "user",
        "content": message.content if isinstance(message, Message) else message,
    }
    messages.append(user_message)


def add_assistant_message(messages, message):
    assistant_message = {
        "role": "assistant",
        "content": message.content if isinstance(message, Message) else message,
    }
    messages.append(assistant_message)


def chat(
    messages,
    system=None,
    temperature=1.0,
    stop_sequences=[],
    tools=None,
    thinking=False,
    thinking_budget=1024,
):
    params = {
        "model": model,
        "max_tokens": 4000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if thinking:
        params["thinking"] = {
            "type": "enabled",
            "budget_tokens": thinking_budget,
        }

    if tools:
        params["tools"] = tools

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message


def text_from_message(message):
    return "\n".join([block.text for block in message.content if block.type == "text"])

In [3]:
# Fire risk assessment prompt
prompt = """
Analyze the attached satellite image of a property with these specific steps:

1. Residence identification: Locate the primary residence on the property by looking for:
   - The largest roofed structure 
   - Typical residential features (driveway connection, regular geometry)
   - Distinction from other structures (garages, sheds, pools)
   Describe the residence's location relative to property boundaries and other features.

2. Tree overhang analysis: Examine all trees near the primary residence:
   - Identify any trees whose canopy extends directly over any portion of the roof
   - Estimate the percentage of roof covered by overhanging branches (0-25%, 25-50%, 50-75%, 75-100%)
   - Note particularly dense areas of overhang

3. Fire risk assessment: For any overhanging trees, evaluate:
   - Potential wildfire vulnerability (ember catch points, continuous fuel paths to structure)
   - Proximity to chimneys, vents, or other roof openings if visible
   - Areas where branches create a "bridge" between wildland vegetation and the structure
   
4. Defensible space identification: Assess the property's overall vegetative structure:
   - Identify if trees connect to form a continuous canopy over or near the home
   - Note any obvious fuel ladders (vegetation that can carry fire from ground to tree to roof)

5. Fire risk rating: Based on your analysis, assign a Fire Risk Rating from 1-4:
   - Rating 1 (Low Risk): No tree branches overhanging the roof, good defensible space around the structure
   - Rating 2 (Moderate Risk): Minimal overhang (<25% of roof), some separation between tree canopies
   - Rating 3 (High Risk): Significant overhang (25-50% of roof), connected tree canopies, multiple points of vulnerability
   - Rating 4 (Severe Risk): Extensive overhang (>50% of roof), dense vegetation against structure, numerous ember catch points, limited defensible space

For each item above (1-5), write one sentence summarizing your findings, with your final response being the numeric Fire Risk Rating (1-4) with a brief justification.
"""

In [4]:
# TODO: Read image data, feed into Claude

with open(r"images/prop1.png", "rb") as f:
    image_bytes = base64.standard_b64encode(f.read()).decode("utf-8")

messages = []

add_user_message(messages, [
                 {
                    "type": "image",
                  "source": {
                      "type": "base64",
                      "media_type": "image/png",
                      "data": image_bytes
                        }
                  },
                  {
                      "type": "text",
                      "text": prompt
                  }
            ])

chat(messages)


Message(id='msg_01L8Pocg4E3tyqzvDZHVudwW', content=[TextBlock(citations=None, text='# Satellite Image Fire Risk Analysis\n\n**1. Residence Identification:**\nThe primary residence is located in the center of the image, identifiable by its tan/gray rectangular roofed structure with what appears to be a darker roof section, positioned along a dirt or gravel driveway/clearing that provides access from the surrounding vegetated area.\n\n**2. Tree Overhang Analysis:**\nMultiple large tree canopies directly overhang substantial portions of the residence roof, with dense foliage coverage appearing to extend over approximately 50-75% of the structure, particularly concentrated on the northern and eastern sections of the building.\n\n**3. Fire Risk Assessment:**\nThe overhanging trees create multiple critical vulnerabilities including direct ember catch points on the roof surface, continuous fuel pathways from surrounding forest vegetation to the structure, and what appears to be direct contact

Read PDF

In [5]:
# TODO: Read image data, feed into Claude

with open(r"earth.pdf", "rb") as f:
    file_bytes = base64.standard_b64encode(f.read()).decode("utf-8")

messages = []

add_user_message(messages, [
                 {
                    "type": "document",
                  "source": {
                      "type": "base64",
                      "media_type": "application/pdf",
                      "data": file_bytes
                        }
                  },
                  {
                      "type": "text",
                      "text": "Summarize the contents of the attached document in 1 sentence."
                  }
            ])

chat(messages)


Message(id='msg_01CLWeZ6skgD5JiZBc4ZTCsM', content=[TextBlock(citations=None, text='This Wikipedia article provides comprehensive information about Earth, including its physical characteristics, orbital properties, formation approximately 4.5 billion years ago, atmosphere composition, status as the only known planet harboring life, and its role as an ocean world with liquid surface water covering 70.8% of its crust.', type='text')], model='claude-sonnet-4-5-20250929', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation=CacheCreation(ephemeral_1h_input_tokens=0, ephemeral_5m_input_tokens=0), cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=9632, output_tokens=66, server_tool_use=None, service_tier='standard'))